# 🔍 FLIR+SCD41 Model Diagnostics and Fixes

This notebook demonstrates how to diagnose and fix common machine learning problems:
1. Model not learning
2. Overfitting (remembering patterns instead of learning)

## Common Issues and Solutions

### Model Not Learning
- **Symptoms**: Poor performance on both training and validation sets
- **Causes**: 
  - Insufficient model capacity
  - Poor data quality
  - Inappropriate learning rate
  - Feature scaling issues

### Overfitting
- **Symptoms**: High performance on training set, poor performance on validation/test sets
- **Causes**:
  - Model too complex for the data
  - Insufficient regularization
  - Too little training data
  - Data leakage

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve, validation_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## 1. 🔄 Generate Sample Data

Create synthetic data for demonstration purposes

In [ ]:
# Generate synthetic dataset
print("🔄 Generating synthetic dataset for demonstration...")

# Generate data using numpy for simplicity
np.random.seed(42)
num_samples = 2000  # Smaller dataset for demo

# Generate features
X = np.random.randn(num_samples, 18)

# Create labels with a clear pattern (first 5 features are important)
y_prob = 1 / (1 + np.exp(-(
    2*X[:, 0] + 1.5*X[:, 1] + X[:, 2] + 
    0.5*X[:, 3] + 0.3*X[:, 4] + 
    0.1*np.sum(X[:, 5:], axis=1) + 
    np.random.normal(0, 0.1, num_samples)
)))
y = (y_prob > 0.5).astype(int)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"✅ Generated {num_samples} samples with 18 features")
print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Class distribution - Train: {np.bincount(y_train)}")
print(f"Class distribution - Validation: {np.bincount(y_val)}")
print(f"Class distribution - Test: {np.bincount(y_test)}")

## 2. 🔍 Diagnose Model Not Learning

Train a model that's too simple and diagnose the issue

In [ ]:
# Example of a model that's too simple (underfitting)
print("🧪 Example: Model that's too simple (underfitting)")

# Create a very simple model (too simple for the data)
simple_model = xgb.XGBClassifier(
    n_estimators=10,      # Very few trees
    max_depth=1,          # Very shallow trees
    learning_rate=0.01,   # Very low learning rate
    random_state=42
)

# Train the model
simple_model.fit(X_train, y_train)

# Evaluate on train and test sets
train_pred = simple_model.predict(X_train)
val_pred = simple_model.predict(X_val)

train_acc = accuracy_score(y_train, train_pred)
val_acc = accuracy_score(y_val, val_pred)

print(f"Simple Model Performance:")
print(f"  Training Accuracy: {train_acc:.4f}")
print(f"  Validation Accuracy: {val_acc:.4f}")
print(f"  Gap: {abs(train_acc - val_acc):.4f}")

# Diagnosis
if train_acc < 0.7 and val_acc < 0.7:
    print("\n🤔 Diagnosis: Model is not learning properly (underfitting)")
    print("💡 Solutions:")
    print("   1. Increase model complexity (more trees, deeper trees)")
    print("   2. Increase learning rate")
    print("   3. Add more relevant features")
    print("   4. Try a different algorithm")
else:
    print("\n✅ Model appears to be learning properly")

## 3. 🔍 Diagnose Overfitting

Train a model that's too complex and diagnose overfitting

In [ ]:
# Example of a model that's too complex (overfitting)
print("🧪 Example: Model that's too complex (overfitting)")

# Create a very complex model
complex_model = xgb.XGBClassifier(
    n_estimators=1000,    # Many trees
    max_depth=10,         # Deep trees
    learning_rate=0.3,    # High learning rate
    subsample=1.0,        # No subsampling
    colsample_bytree=1.0, # No column subsampling
    reg_alpha=0,          # No L1 regularization
    reg_lambda=0,         # No L2 regularization
    random_state=42
)

# Train the model
complex_model.fit(X_train, y_train)

# Evaluate on train and test sets
train_pred = complex_model.predict(X_train)
val_pred = complex_model.predict(X_val)

train_acc = accuracy_score(y_train, train_pred)
val_acc = accuracy_score(y_val, val_pred)

print(f"Complex Model Performance:")
print(f"  Training Accuracy: {train_acc:.4f}")
print(f"  Validation Accuracy: {val_acc:.4f}")
print(f"  Gap: {train_acc - val_acc:.4f}")

# Diagnosis
if (train_acc - val_acc) > 0.1:
    print("\n⚠️  Diagnosis: Model is overfitting (remembering patterns)")
    print("💡 Solutions:")
    print("   1. Add regularization (reg_alpha, reg_lambda)")
    print("   2. Reduce model complexity (max_depth, n_estimators)")
    print("   3. Use early stopping")
    print("   4. Add dropout or other regularization techniques")
    print("   5. Get more training data")
    print("   6. Use cross-validation for better evaluation")
else:
    print("\n✅ Model does not appear to be overfitting")

## 4. 📈 Learning Curves Visualization

Use learning curves to diagnose bias and variance problems

In [ ]:
# Generate learning curves
print("📊 Generating learning curves...")

# Simple model learning curve
simple_model_lc = xgb.XGBClassifier(
    n_estimators=10,
    max_depth=1,
    learning_rate=0.01,
    random_state=42
)

train_sizes, train_scores, val_scores = learning_curve(
    simple_model_lc, X_train, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy'
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 5))

# Simple model
plt.subplot(1, 2, 1)
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('Learning Curves - Simple Model (Underfitting)')
plt.legend(loc='best')
plt.grid(True)

# Complex model
complex_model_lc = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=10,
    learning_rate=0.3,
    random_state=42
)

train_sizes, train_scores, val_scores = learning_curve(
    complex_model_lc, X_train, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy'
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.subplot(1, 2, 2)
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy Score')
plt.title('Learning Curves - Complex Model (Overfitting)')
plt.legend(loc='best')
plt.grid(True)

plt.tight_layout()
plt.show()

print("Learning curves help diagnose:")
print("1. High bias (underfitting): Both curves converge to low scores")
print("2. High variance (overfitting): Large gap between curves")
print("3. Good fit: Both curves converge to high scores with small gap")

## 5. 📈 Validation Curves

Use validation curves to find optimal hyperparameters

In [ ]:
# Generate validation curves
print("📊 Generating validation curves...")

# Validation curve for max_depth
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

train_scores, val_scores = validation_curve(
    model, X_train, y_train,
    param_name='max_depth', param_range=range(1, 11),
    cv=5, scoring='accuracy', n_jobs=-1
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot validation curves
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(range(1, 11), train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(range(1, 11), val_mean, 'o-', color='red', label='Validation score')
plt.fill_between(range(1, 11), val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy Score')
plt.title('Validation Curves - Max Depth')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Find optimal parameter
optimal_idx = np.argmax(val_mean)
optimal_max_depth = range(1, 11)[optimal_idx]
print(f"Optimal max_depth: {optimal_max_depth}")
print(f"Best validation score: {val_mean[optimal_idx]:.4f}")

## 6. ✅ Properly Tuned Model

Train a model with proper regularization to prevent overfitting

In [ ]:
# Train a well-tuned model with regularization
print("✅ Training a well-tuned model with regularization...")

# Well-tuned model with regularization
tuned_model = xgb.XGBClassifier(
    n_estimators=200,         # Reasonable number of trees
    max_depth=4,              # Moderate depth
    learning_rate=0.1,        # Moderate learning rate
    subsample=0.8,            # Subsampling to prevent overfitting
    colsample_bytree=0.8,     # Column subsampling
    reg_alpha=0.1,            # L1 regularization
    reg_lambda=1.0,           # L2 regularization
    random_state=42
)

# Train the model
tuned_model.fit(X_train, y_train)

# Evaluate on all sets
train_pred = tuned_model.predict(X_train)
val_pred = tuned_model.predict(X_val)
test_pred = tuned_model.predict(X_test)

train_acc = accuracy_score(y_train, train_pred)
val_acc = accuracy_score(y_val, val_pred)
test_acc = accuracy_score(y_test, test_pred)

print(f"Well-Tuned Model Performance:")
print(f"  Training Accuracy: {train_acc:.4f}")
print(f"  Validation Accuracy: {val_acc:.4f}")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"  Train-Val Gap: {train_acc - val_acc:.4f}")

# Diagnosis
if train_acc > 0.8 and val_acc > 0.8 and (train_acc - val_acc) < 0.1:
    print("\n🎉 Success: Model is well-tuned and properly learning!")
    print("   - Good performance on training set")
    print("   - Good performance on validation set")
    print("   - Small gap between training and validation")
    print("   - Generalizes well to test set")
else:
    print("\n⚠️  Model may still need tuning")

## 7. 🧠 Neural Network Overfitting Example

Demonstrate overfitting and regularization in neural networks

In [ ]:
# Simple neural network without regularization (prone to overfitting)
print("🧠 Neural Network Overfitting Example")

class SimpleNN(nn.Module):
    def __init__(self, input_size=18, hidden_size=128):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Convert data to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)

# Create model
device = torch.device('cpu')
simple_nn = SimpleNN().to(device)

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(simple_nn.parameters(), lr=0.001)

# Training loop (no regularization, prone to overfitting)
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(100):
    # Training
    simple_nn.train()
    optimizer.zero_grad()
    outputs = simple_nn(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # Calculate training accuracy
    train_pred = torch.argmax(outputs, dim=1)
    train_acc = accuracy_score(y_train_tensor, train_pred)
    
    # Validation
    simple_nn.eval()
    with torch.no_grad():
        val_outputs = simple_nn(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_pred = torch.argmax(val_outputs, dim=1)
        val_acc = accuracy_score(y_val_tensor, val_pred)
    
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    # Print progress every 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/100]')
        print(f'  Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc:.4f}')

# Plot training and validation curves
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves - Simple NN (Overfitting)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves - Simple NN (Overfitting)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Check for overfitting
final_train_acc = train_accuracies[-1]
final_val_acc = val_accuracies[-1]
gap = final_train_acc - final_val_acc

print(f"\nSimple NN Final Performance:")
print(f"  Training Accuracy: {final_train_acc:.4f}")
print(f"  Validation Accuracy: {final_val_acc:.4f}")
print(f"  Gap: {gap:.4f}")

if gap > 0.1:
    print("\n⚠️  Diagnosis: Neural Network is overfitting!")
    print("💡 Solutions:")
    print("   1. Add dropout layers")
    print("   2. Add L1/L2 regularization")
    print("   3. Reduce network complexity")
    print("   4. Use early stopping")
    print("   5. Data augmentation")
else:
    print("\n✅ Neural Network does not appear to be overfitting")

## 8. ✅ Regularized Neural Network

Train a neural network with proper regularization

In [ ]:
# Regularized neural network with dropout and L2 regularization
print("✅ Regularized Neural Network Example")

class RegularizedNN(nn.Module):
    def __init__(self, input_size=18, hidden_sizes=[64, 32], dropout_rate=0.3):
        super(RegularizedNN, self).__init__()
        layers = []
        prev_size = input_size
        
        # Hidden layers with dropout
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        # Output layer
        layers.append(nn.Linear(prev_size, 2))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Create model with regularization
regularized_nn = RegularizedNN(dropout_rate=0.3).to(device)

# Training setup with L2 regularization
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(regularized_nn.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization

# Training loop with early stopping
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(100):
    # Training
    regularized_nn.train()
    optimizer.zero_grad()
    outputs = regularized_nn(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # Calculate training accuracy
    train_pred = torch.argmax(outputs, dim=1)
    train_acc = accuracy_score(y_train_tensor, train_pred)
    
    # Validation
    regularized_nn.eval()
    with torch.no_grad():
        val_outputs = regularized_nn(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_pred = torch.argmax(val_outputs, dim=1)
        val_acc = accuracy_score(y_val_tensor, val_pred)
    
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Print progress every 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{100}]')
        print(f'  Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc:.4f}')

# Plot training and validation curves
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves - Regularized NN')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves - Regularized NN')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Check for overfitting
final_train_acc = train_accuracies[-1]
final_val_acc = val_accuracies[-1]
gap = final_train_acc - final_val_acc

print(f"\nRegularized NN Final Performance:")
print(f"  Training Accuracy: {final_train_acc:.4f}")
print(f"  Validation Accuracy: {final_val_acc:.4f}")
print(f"  Gap: {gap:.4f}")

if gap <= 0.1:
    print("\n🎉 Success: Regularized Neural Network is properly learning!")
    print("   - Good performance on training set")
    print("   - Good performance on validation set")
    print("   - Small gap between training and validation")
    print("   - Early stopping prevented overfitting")
    print("   - Dropout and L2 regularization helped generalization")
else:
    print("\n⚠️  Neural Network may still be overfitting")

## 9. 🎯 Key Takeaways

### Detecting Learning Issues
1. **Compare training and validation performance**
   - Large gap = overfitting
   - Poor performance on both = underfitting

2. **Use learning curves**
   - Converge to low scores = underfitting
   - Large gap = overfitting
   - Converge to high scores with small gap = good fit

3. **Use validation curves**
   - Find optimal hyperparameters
   - Identify when increasing complexity hurts performance

### Preventing Overfitting
1. **Regularization**
   - L1/L2 regularization
   - Dropout layers
   - Early stopping

2. **Model Architecture**
   - Reduce model complexity
   - Use appropriate depth/width

3. **Data Techniques**
   - Data augmentation
   - Cross-validation
   - More training data

### Ensuring Proper Learning
1. **Data Quality**
   - Check for data leakage
   - Ensure sufficient variance in features
   - Balance class distributions

2. **Model Configuration**
   - Appropriate learning rate
   - Sufficient training time
   - Proper feature scaling

3. **Evaluation**
   - Use separate test set
   - Cross-validation for robust estimates
   - Multiple metrics (accuracy, F1, precision, recall)

For the FLIR+SCD41 fire detection system, these techniques are implemented in:
- `scripts/improved_flir_scd41_training.py`
- `scripts/model_diagnostics.py`